#Movie Recommendation System

In this project, we are going to create a movie recommendation system using Content based filtering technique. Content based filtering considers the movie properties/meta data such as Genre, Story Overview, Cast,etc. and recommend accordingly.

# Importing necessary libraries

In [7]:
pip install sentence-transformers -q

In [8]:
import pandas as pd
from  ast import literal_eval
import gensim.downloader as api
from nltk.tokenize import word_tokenize
import numpy as np
import nltk
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Importing Data

In [9]:
df_movie = pd.read_csv('tmdb_5000_movies.csv')
df_movie.head(5)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [10]:
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

## Text Pre-processing

Removing duplicates

In [11]:
df_movie['title'].value_counts().sort_values(ascending=False)

title
The Host                     2
Out of the Blue              2
Batman                       2
Drag Me to Hell              1
The Dark Knight Rises        1
                            ..
Newlyweds                    1
Signed, Sealed, Delivered    1
Shanghai Calling             1
Breaking Upwards             1
My Date with Drew            1
Name: count, Length: 4800, dtype: int64

In [12]:
df_movie.drop_duplicates(subset = ['title'],inplace=True)

In [13]:
df_movie.isnull().sum()

budget                     0
genres                     0
homepage                3089
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [14]:
df_movie['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [15]:
df_movie['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

The columns keywords and genre are in the string format. Let's convert them into objects

In [16]:
df_movie['keywords']=df_movie['keywords'].apply(literal_eval)

In [17]:
df_movie['keywords'][0]

[{'id': 1463, 'name': 'culture clash'},
 {'id': 2964, 'name': 'future'},
 {'id': 3386, 'name': 'space war'},
 {'id': 3388, 'name': 'space colony'},
 {'id': 3679, 'name': 'society'},
 {'id': 3801, 'name': 'space travel'},
 {'id': 9685, 'name': 'futuristic'},
 {'id': 9840, 'name': 'romance'},
 {'id': 9882, 'name': 'space'},
 {'id': 9951, 'name': 'alien'},
 {'id': 10148, 'name': 'tribe'},
 {'id': 10158, 'name': 'alien planet'},
 {'id': 10987, 'name': 'cgi'},
 {'id': 11399, 'name': 'marine'},
 {'id': 13065, 'name': 'soldier'},
 {'id': 14643, 'name': 'battle'},
 {'id': 14720, 'name': 'love affair'},
 {'id': 165431, 'name': 'anti war'},
 {'id': 193554, 'name': 'power relations'},
 {'id': 206690, 'name': 'mind and soul'},
 {'id': 209714, 'name': '3d'}]

In [18]:
df_movie['genres']=df_movie['genres'].apply(literal_eval)

In [19]:
df_movie['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

Let's take the value of the name inside each row of the column it has the genre and keywords

In [20]:
keywords = []
for i in df_movie['keywords']:
  keys =[]
  for j in i:
    keys.append(j['name'])
  keywords.append(keys)

In [21]:
df_movie['keywords_new']=keywords

In [22]:
keywords[0]

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [23]:
genre = []
for i in df_movie['genres']:
  g=[]
  for j in i:
    g.append(j['name'])
  genre.append(g)

In [24]:
df_movie['genres_new']=genre

In [25]:
df_movie.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,keywords_new,genres_new
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]"
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://disney.go.com/disneypictures/pirates/,285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]"
2,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]"
3,250000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",http://www.thedarkknightrises.com/,49026,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]"
4,260000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://movies.disney.com/john-carter,49529,"[{'id': 818, 'name': 'based on novel'}, {'id':...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]"


In [26]:
movie = pd.DataFrame({'title':df_movie['title'],'keywords':keywords,'genre':genre,'overview':df_movie['overview'],})

In [27]:
movie

,title,keywords,genre,overview
0,Avatar,"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha..."
2,Spectre,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...
4,John Carter,"[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca..."
...,...,...,...,...
4798,El Mariachi,"[united states–mexico barrier, legs, arms, pap...","[Action, Crime, Thriller]",El Mariachi just wants to play his guitar and ...
4799,Newlyweds,[],"[Comedy, Romance]",A newlywed couple's honeymoon is upended by th...
4800,"Signed, Sealed, Delivered","[date, love at first sight, narration, investi...","[Comedy, Drama, Romance, TV Movie]","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,Shanghai Calling,[],[],When ambitious New York attorney Sam is sent t...


In [28]:
movie.isna().sum()

title       0
keywords    0
genre       0
overview    3
dtype: int64

In [29]:
movie.dropna(inplace=True)

In [30]:
punc = string.punctuation
punc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [31]:
lemma = WordNetLemmatizer()

In [32]:
stop_words = stopwords.words('english')

In [33]:
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [34]:
movie['genre']= movie['genre'].apply(lambda x:' '.join(x) )

In [35]:
movie['genre']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genre, Length: 4797, dtype: object

In [36]:
movie['keywords'] = movie['keywords'].apply(lambda x:' '.join(x))

In [37]:
movie['keywords']

0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6 bri...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
                              ...                        
4798    united states–mexico barrier legs arms paper k...
4799                                                     
4800    date love at first sight narration investigati...
4801                                                     
4802                 obsession camcorder crush dream girl
Name: keywords, Length: 4797, dtype: object

We will be doing the text preprocessing steps such as stop word removal, punctuation removal, lemmatization and tokeniztion

In [38]:
def tokenizer(input_text):
  tokens = nltk.word_tokenize(input_text)
  tokens = [lemma.lemmatize(token.lower()) for token in tokens]
  tokens = [token for token in tokens if token not in punc and token not in stop_words]
  return tokens


In [71]:
movie['overview_tokens'] = movie['overview'].apply(tokenizer)

In [72]:
movie['overview_tokens']

0       [22nd, century, paraplegic, marine, dispatched...
1       [captain, barbossa, long, believed, dead, ha, ...
2       [cryptic, message, bond, ’, past, sends, trail...
3       [following, death, district, attorney, harvey,...
4       [john, carter, war-weary, former, military, ca...
                              ...                        
4798    [el, mariachi, want, play, guitar, carry, fami...
4799    [newlywed, couple, 's, honeymoon, upended, arr...
4800    [``, signed, sealed, delivered, '', introduces...
4801    [ambitious, new, york, attorney, sam, sent, sh...
4802    [ever, since, second, grade, first, saw, e.t, ...
Name: overview_tokens, Length: 4797, dtype: object

In [41]:
movie['genre_tokens'] = movie['genre'].apply(tokenizer)

In [42]:
movie['genre_tokens']

0       [action, adventure, fantasy, science, fiction]
1                         [adventure, fantasy, action]
2                           [action, adventure, crime]
3                     [action, crime, drama, thriller]
4                [action, adventure, science, fiction]
                             ...                      
4798                         [action, crime, thriller]
4799                                 [comedy, romance]
4800               [comedy, drama, romance, tv, movie]
4801                                                []
4802                                     [documentary]
Name: genre_tokens, Length: 4797, dtype: object

In [43]:
movie['keyword_token'] = movie['keywords'].apply(tokenizer)

In [44]:
movie['keyword_token']

0       [culture, clash, future, space, war, space, co...
1       [ocean, drug, abuse, exotic, island, east, ind...
2       [spy, based, novel, secret, agent, sequel, mi6...
3       [dc, comic, crime, fighter, terrorist, secret,...
4       [based, novel, mar, medallion, space, travel, ...
                              ...                        
4798    [united, states–mexico, barrier, leg, arm, pap...
4799                                                   []
4800    [date, love, first, sight, narration, investig...
4801                                                   []
4802           [obsession, camcorder, crush, dream, girl]
Name: keyword_token, Length: 4797, dtype: object

In [80]:
movie

,title,keywords,genre,overview,tokens,genre_tokens,keyword_token,overview_tokens
0,Avatar,culture clash future space war space colony so...,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...","[22nd, century, paraplegic, marine, dispatched...","[action, adventure, fantasy, science, fiction]","[culture, clash, future, space, war, space, co...","[22nd, century, paraplegic, marine, dispatched..."
1,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...","[captain, barbossa, long, believed, dead, ha, ...","[adventure, fantasy, action]","[ocean, drug, abuse, exotic, island, east, ind...","[captain, barbossa, long, believed, dead, ha, ..."
2,Spectre,spy based on novel secret agent sequel mi6 bri...,Action Adventure Crime,A cryptic message from Bond’s past sends him o...,"[cryptic, message, bond, ’, past, sends, trail...","[action, adventure, crime]","[spy, based, novel, secret, agent, sequel, mi6...","[cryptic, message, bond, ’, past, sends, trail..."
3,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Action Crime Drama Thriller,Following the death of District Attorney Harve...,"[following, death, district, attorney, harvey,...","[action, crime, drama, thriller]","[dc, comic, crime, fighter, terrorist, secret,...","[following, death, district, attorney, harvey,..."
4,John Carter,based on novel mars medallion space travel pri...,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...","[john, carter, war-weary, former, military, ca...","[action, adventure, science, fiction]","[based, novel, mar, medallion, space, travel, ...","[john, carter, war-weary, former, military, ca..."
...,...,...,...,...,...,...,...,...
4798,El Mariachi,united states–mexico barrier legs arms paper k...,Action Crime Thriller,El Mariachi just wants to play his guitar and ...,"[el, mariachi, want, play, guitar, carry, fami...","[action, crime, thriller]","[united, states–mexico, barrier, leg, arm, pap...","[el, mariachi, want, play, guitar, carry, fami..."
4799,Newlyweds,,Comedy Romance,A newlywed couple's honeymoon is upended by th...,"[newlywed, couple, 's, honeymoon, upended, arr...","[comedy, romance]",[],"[newlywed, couple, 's, honeymoon, upended, arr..."
4800,"Signed, Sealed, Delivered",date love at first sight narration investigati...,Comedy Drama Romance TV Movie,"""Signed, Sealed, Delivered"" introduces a dedic...","[``, signed, sealed, delivered, '', introduces...","[comedy, drama, romance, tv, movie]","[date, love, first, sight, narration, investig...","[``, signed, sealed, delivered, '', introduces..."
4801,Shanghai Calling,,,When ambitious New York attorney Sam is sent t...,"[ambitious, new, york, attorney, sam, sent, sh...",[],[],"[ambitious, new, york, attorney, sam, sent, sh..."


## Creating Recommendation System

In [45]:
def get_recommendation(title,similarity_score):
  idx = movie[movie['title'] == title].index[0]
  score = list(enumerate(similarity_score[idx]))
  sorted_score = sorted(score,key = lambda x:x[1],reverse = True)
  sorted_score = sorted_score[1:11]
  sorted_idx = [i[0] for i in sorted_score]
  recommendations = movie.iloc[sorted_idx]['title']
  return recommendations


To get the similarity score between movies, we need to get the vectors(numeric form of text) of the tokens we have generated.

### Bag of Words

Bag of words is a vectorizing technique where each word/token will be converted into numbers based on the frequency in the entire corpus

In [46]:
count = CountVectorizer(stop_words='english')

In [47]:

overview_matrix = count.fit_transform(movie['overview_tokens'].apply(lambda x:' '.join(x)))


In [48]:
overview_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

We would be using cosine similarity technique to find the similarity between the vector representation of the movies

In [49]:

sim = cosine_similarity(overview_matrix,overview_matrix)

In [50]:
sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.0761387 , 0.        ,
        0.03892495],
       [0.        , 0.        , 1.        , ..., 0.02948839, 0.        ,
        0.        ],
       ...,
       [0.        , 0.0761387 , 0.02948839, ..., 1.        , 0.04347826,
        0.02222771],
       [0.        , 0.        , 0.        , ..., 0.04347826, 1.        ,
        0.04445542],
       [0.        , 0.03892495, 0.        , ..., 0.02222771, 0.04445542,
        1.        ]])

In [51]:
genre_matrix = count.fit_transform(movie['genre_tokens'].apply(lambda x:' '.join(x)))

In [52]:
genre_matrix.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [53]:
sim_1 = cosine_similarity(genre_matrix,genre_matrix)

In [54]:
sim_1

array([[1.        , 0.77459667, 0.51639778, ..., 0.        , 0.        ,
        0.        ],
       [0.77459667, 1.        , 0.66666667, ..., 0.        , 0.        ,
        0.        ],
       [0.51639778, 0.66666667, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [55]:
keyword_matrix = count.fit_transform(movie['keyword_token'].apply(lambda x:' '.join(x)))

In [56]:
sim_2 = cosine_similarity(keyword_matrix,keyword_matrix)

Let's see how each feature similarity scores are giving recommendatiions

In [57]:
get_recommendation('The Avengers',sim)

2136             Team America: World Police
4293                           The Algerian
256                               Allegiant
673             Austin Powers in Goldmember
3751           What the #$*! Do We (K)now!?
154                   Rise of the Guardians
2227                     Battle of the Year
3209                             Skin Trade
1706                                Miracle
1769    Spy Kids: All the Time in the World
Name: title, dtype: object

In [58]:
get_recommendation('The Avengers',sim_1)

7                 Avengers: Age of Ultron
16                           The Avengers
26             Captain America: Civil War
31                             Iron Man 3
35    Transformers: Revenge of the Fallen
36        Transformers: Age of Extinction
39                           TRON: Legacy
47                Star Trek Into Darkness
51                            Pacific Rim
52         Transformers: Dark of the Moon
Name: title, dtype: object

In [59]:
get_recommendation('The Avengers',sim_2)

182                                Ant-Man
7                  Avengers: Age of Ultron
79                              Iron Man 2
85     Captain America: The Winter Soldier
126                   Thor: The Dark World
26              Captain America: Civil War
129                                   Thor
203                                     X2
511                                  X-Men
169     Captain America: The First Avenger
Name: title, dtype: object

Lets get the recommendation based on the averge similarity score of all 3 featues

In [60]:
count_sim = (sim + sim_1 + sim_2)/3

In [61]:
get_recommendation('The Avengers',count_sim)

7                  Avengers: Age of Ultron
182                                Ant-Man
85     Captain America: The Winter Soldier
79                              Iron Man 2
26              Captain America: Civil War
169     Captain America: The First Avenger
511                                  X-Men
242                         Fantastic Four
174                    The Incredible Hulk
68                                Iron Man
Name: title, dtype: object

## Sentence Transformers

SBERT (Sentence Transformers) is an open source transformer model which is suitable for sentence similarity tasks

In [62]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

since SBERT can perform well with raw text (having builtin pre-processing methods), let's use

In [77]:
sbert_overview = movie['overview'].apply(model.encode)

In [70]:
len(model.encode("Helllo how are you"))

384

In [78]:
sbert_sim_1 = cosine_similarity(sbert_overview.tolist(),sbert_overview.tolist())

In [79]:
get_recommendation('The Avengers',sbert_sim_1)

7                   Avengers: Age of Ultron
85      Captain America: The Winter Soldier
456                                    Fury
392                              Safe House
595                                    Doom
26               Captain America: Civil War
3505                             Men of War
3098                              War, Inc.
3306                          Code of Honor
3209                             Skin Trade
Name: title, dtype: object

In [81]:
sbert_keyword = movie['keywords'].apply(model.encode)

In [82]:
sbert_sim_2 = cosine_similarity(sbert_keyword.tolist(),sbert_keyword.tolist())

In [83]:
get_recommendation('The Avengers',sbert_sim_2)

85     Captain America: The Winter Soldier
182                                Ant-Man
79                              Iron Man 2
169     Captain America: The First Avenger
129                                   Thor
14                            Man of Steel
7                  Avengers: Age of Ultron
126                   Thor: The Dark World
788                               Deadpool
41                           Green Lantern
Name: title, dtype: object

In [84]:
sbert_genre = movie['genre'].apply(model.encode)

In [85]:
sbert_sim_3 = cosine_similarity(sbert_genre.tolist(),sbert_genre.tolist())

In [86]:
get_recommendation('The Avengers',sbert_sim_3)

35      Transformers: Revenge of the Fallen
36          Transformers: Age of Extinction
158                               Star Trek
174                     The Incredible Hulk
182                                 Ant-Man
193                             After Earth
260                            Ender's Game
2492                                  Steel
51                              Pacific Rim
52           Transformers: Dark of the Moon
Name: title, dtype: object

Averaging the similarity score and find the recommendation

In [87]:
sbert_sim = (sbert_sim_1 + sbert_sim_2 + sbert_sim_3)/3

In [88]:
get_recommendation('The Avengers',sbert_sim)

85     Captain America: The Winter Soldier
7                  Avengers: Age of Ultron
182                                Ant-Man
174                    The Incredible Hulk
26              Captain America: Civil War
68                                Iron Man
79                              Iron Man 2
169     Captain America: The First Avenger
31                              Iron Man 3
14                            Man of Steel
Name: title, dtype: object

### Word2vec

Word2vec is a vectorizing technique where similar words will be close to each other in the vector/feature space

In [ ]:

'''Using Google Pretrained Word2vec model'''

# Load the pre-trained Word2Vec model
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
#model = KeyedVectors.load('google-word2vec.kv')

In [ ]:
def sentence_vector(words, model, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0.0

    for word in words:
        if word in model:
            num_words += 1
            feature_vector = np.add(feature_vector, model[word])

    if num_words > 0:
        feature_vector = np.divide(feature_vector, num_words)

    return feature_vector

In [ ]:

vectorized_text = movie['overview_tokens'].apply(sentence_vector,args = [model, 300])

In [ ]:
vectorized_text

0       [0.036981203, 0.10019124, 0.0035644532, 0.0362...
1       [0.036849976, 0.06332588, 0.05958557, 0.075546...
2       [0.08772217, 0.030266114, 0.032296143, -0.0131...
3       [0.013479324, 0.064304896, 0.018707458, 0.0003...
4       [0.08323387, 0.09370931, -0.011795609, 0.07106...
                              ...                        
4798    [0.066419035, 0.011448216, 0.055078454, 0.1041...
4799    [-0.009800502, 0.0055672782, -0.19485365, 0.01...
4800    [0.015217271, 0.06475694, -0.00018363775, 0.02...
4801    [-0.016468769, 0.04645813, -0.05894594, 0.0280...
4802    [0.03533099, 0.061220232, 0.062316403, 0.05566...
Name: tokens, Length: 4797, dtype: object

In [ ]:
word_sim = cosine_similarity(vectorized_text.to_list(),vectorized_text.to_list())

In [ ]:
genre_vector = movie['genre_tokens'].apply(sentence_vector,args = [model, 300])

In [ ]:
word_sim_1 = cosine_similarity(genre_vector.tolist(),genre_vector.tolist())

In [ ]:
keyword_vector = movie['keyword_token'].apply(sentence_vector,args =[model,300])

In [ ]:
word_sim_2 = cosine_similarity(keyword_vector.tolist(),keyword_vector.tolist())

In [ ]:
get_recommendation('The Avengers',word_sim)

1234                          The Art of War
7                    Avengers: Age of Ultron
531                  The Man from U.N.C.L.E.
153     Mission: Impossible - Ghost Protocol
657               Resident Evil: Retribution
91              Independence Day: Resurgence
97                             Shin Godzilla
47                   Star Trek Into Darkness
51                               Pacific Rim
1892                              The Losers
Name: title, dtype: object

In [ ]:
get_recommendation('The Avengers',word_sim_1)

7                 Avengers: Age of Ultron
16                           The Avengers
26             Captain America: Civil War
31                             Iron Man 3
35    Transformers: Revenge of the Fallen
36        Transformers: Age of Extinction
39                           TRON: Legacy
47                Star Trek Into Darkness
51                            Pacific Rim
52         Transformers: Dark of the Moon
Name: title, dtype: object

In [ ]:
get_recommendation('The Avengers',word_sim_2)

7                  Avengers: Age of Ultron
129                                   Thor
79                              Iron Man 2
182                                Ant-Man
85     Captain America: The Winter Soldier
126                   Thor: The Dark World
64                       X-Men: Apocalypse
26              Captain America: Civil War
203                                     X2
511                                  X-Men
Name: title, dtype: object

Averaging similarity scores of all 3 features

In [ ]:
word2vec_sim = (word_sim + word_sim_1 + word_sim_2)/3

In [ ]:
get_recommendation('The Avengers',word2vec_sim)

7                  Avengers: Age of Ultron
182                                Ant-Man
85     Captain America: The Winter Soldier
79                              Iron Man 2
242                         Fantastic Four
26              Captain America: Civil War
174                    The Incredible Hulk
511                                  X-Men
31                              Iron Man 3
46              X-Men: Days of Future Past
Name: title, dtype: object

## Conclusion



*   We can clearly see that averaging the similarity scores gives better recommendations and word2vec is little better than SBERT in this context.
*   May be we shall try using more features and SBERT might preform better at that point

